# Gentle introduction to Ray datasets APIs

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives

In this introductory tutorial you will learn to:
 * create, transform, read and save Ray datasets
 * use shards for parallel processing of large datasets
 * understand datapipelines and their merits
 * use DatasetPipeline for last-mile ML ingestion for distributed training

### Overview

This is a brief and gentle introduction to Ray's native library `ray dataset`. As a native Ray library, built atop Ray, it allows you to exchange data among Ray tasks, actors, libraries, and applications. 

Ray Datasets, using distributed Apache Arrow, are designed to load and preprocess data for distributed ML training pipelines. Compared to other loading solutions, Datasets are more flexible (e.g., can express higher-quality per-epoch global shuffles) and provides higher overall performance.

Additionally, Ray datasets provides standard transformations like `map`, `filter`, and `partition`. Ray datasets is *not* a replacement for a full-fledged data processing library EDA, ETL or a subsitute for Apache Spark or Dask or Pandas DataFrames. It's primary objective is last-mile rudimentary data preprocessing and data ingestion for ML training.

Supporting myriad [file formats and data sources](https://docs.ray.io/en/latest/data/dataset.html#datasource-compatibility), you can read from and write to local FS and cloud storage. 

<img src="images/dataset.png" width="70%" height="35%">


### Ray Datasets

A Ray dataset implements a distributed [Apache Arrow](https://arrow.apache.org/). A Dataset consists of a list of Ray object references to blocks. Each block holds a set of items in either an [Arrow table](https://arrow.apache.org/docs/python/data.html#tables) or a Python list (for Arrow incompatible objects).

<img src="images/dataset-arch.png" width="70%" height="35%">

### Creating datasets

In [ ]:
import logging, random
import ray

In [ ]:
if ray.is_initialized:
    ray.shutdown()
ctx = ray.init(logging_level=logging.ERROR)
print(ctx)

In [ ]:
print(f"Dashboard url: http://{ctx.address_info['webui_url']}")

Let's create a generic dataset of 100K integers and look at the schema and underlying datatype. The difference between `show` and `take` is that the former takes one item at time and prints it, while the latter iterates over row items from the dataset, appends to a list and returns it. Underneath, `ds.show()` calls `ds.take()`.

In [ ]:
ds = ray.data.range(100_000)
ds.count(), ds.schema(), ds.show(5), ds.take(5)

Let's create a synthetic dataset of Arrow records with several columns and data associated with it. 

In [ ]:
STATES = ["CA", "AZ", "OR", "WA", "TX", "UT"]
M_STATUS = ["married", "single", "domestic", "divorced", "undeclared"]
GENDER = ["F", "M", "U"]
HOME_OWNER = ["condo", "house", "rental"]

items = [{"id": i,
          "ssn": None,
          "name": None,
          "amount": i * 1.5, 
          "interest": random.randint(1,5) * .1,
          "state": random.choice(STATES),
          "marital_status": random.choice(M_STATUS),
          "property": random.choice(HOME_OWNER),
          "dependents": random.randint(1, 5),
          "defaulted": random.randint(0,1),
          "gender":random.choice(GENDER) } for i in range(1,750_001)]
items[:1]

In [ ]:
arrow_ds = ray.data.from_items(items)
arrow_ds

In [ ]:
arrow_ds.count(), arrow_ds.take(2)

In [ ]:
arrow_ds.schema

### Saving datasets
Ray datasets support myriad data formats and public storage. Let's save this dataset as a parquet file and create 4 partitions

In [ ]:
arrow_ds.repartition(5).write_parquet("data/interest.parquet")

In [ ]:
!ls -l data/interest.parquet

### Transformation

Ray datasets support transformation in parallel using `map`. It uses Ray tasks to execute eagerly or synchronously. Among others [transformations](https://docs.ray.io/en/latest/data/package-ref.html#dataset-api), it supports`filter`, `flat_map`, `groupBy`etc.

Let's try a using `.map()`, `.filter()` and `.groupBy` on our dataset.

In [ ]:
arrow_ds.map(lambda x: x['amount'] * 2.5).take(5)

In [ ]:
arrow_ds.filter(lambda x: x['amount'] > 10000.00 and x['state'] == 'CA').take(2)

In [ ]:
results = arrow_ds.groupby("state").count()

In [ ]:
results.show()

In [ ]:
results=arrow_ds.max(["amount", "interest", "dependents"])
results

### Accessing datasets

Datasets can be passed to Ray tasks or actors and read with `.iter_batches()` or `.iter_rows()`. This does not incur a copy, since the blocks of the Dataset are passed by reference as Ray objects. Splitting data as shards and passing to individual Ray Actors to process shards in a common Ray pattern used in distributed training with Ray actors.

Let's examine how.

*Questions for clearificaiton*:
 * _why is shard a list now instead of Dataset_?

In [ ]:
@ray.remote
class BatchWorker:
    def __init__(self, rank):
        self.rank = rank
        self.processed= 0
    
    @ray.method(num_returns=2)
    def process_shard_list(self, shard: ray.data.Dataset) -> tuple:
        for batch in shard.iter_batches(batch_size=1024):
            # do something with the batch such as feature
            # processing, transformation, and 
            # save as a parquet files 
            self.processed = self.processed + len(batch)
        # return items processed, worker id
        return (self.processed, self.rank)     

#### Create batch workers as Ray actors
Each actor will get a shard, list of rows, to work on. We split
our dataset `arrow_ds` into five shards. Each `BatchWorker` gets a shard.

In [ ]:
batch_workers = [BatchWorker.remote(i) for i in range(1, 6)]

shards = arrow_ds.split(len(batch_workers), locality_hints=batch_workers)

print(f"Shard row: {shards[0]}")
print(f"Number of shards:{len(shards)}")
print(f"Number of shard workers:{len(batch_workers)}")

### Launch `BatchWorker` actors

Process each shard. Each `BatchWorker.process_shard_list()` returns a object RefID with a tuple as its value. What we get from this comprehension is a list objectRefs as tuples.

In [ ]:
object_refs = [w.process_shard_list.remote(s) for w, s in zip(batch_workers, shards)]
# object_refs, len(object_refs)

Fetch the values from the returned list of ObjectRefs, which is a tuple of (batch_size, worker_rank).

In [ ]:
values = [ray.get(ref) for ref in object_refs]
values

### Creating and using dataset pipelines

What are dataset pipelines and how are they different from Ray datasets? 

Datasets perform transformation or operations eagerly or synchronously, whereas [DataPipelines](https://docs.ray.io/en/latest/data/package-ref.html#datasetpipeline-api) can execute in an overlaped pipeline executions. For example, if you had operations that require reading from file, transforming data, and then doing some minor feature engineering, these operations can be executed in a normal pipeline fashion. This allows for the overlapped execution of data input (e.g., reading files), computation (e.g. feature preprocessing), and training (e.g., distributed ML training).

A `DatasetPipeline` can be constructed in two ways: either by pipelining the execution of an existing Dataset (via `Dataset.window`) or generating repeats of an existing Dataset (via `Dataset.repeat`). 

Let's have a go at it and see what we can do with our synthetic data from above.


### Using Dataset.window

Create some functions or operations to be executed in a overlapped manner in the pipeline. These functions
are simple to illustrate a point. But they can be complex for a particular use case.

 _Questions for clarification_:
 * _how can we send arguments to thse pipeline functions?_

In [ ]:
def divide_row_value(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return round(row / 2)

In [ ]:
def double_row_value(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return row * 2

In [ ]:
def modulo_row_value(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return row % random.randint(1, 42)

#### Create a window based pipeline
With a each window of 50 blocks. 

_Questions for clarification_:
 * _why the number of stages is 2?_

In [ ]:
ds_pipe = ds.window(blocks_per_window=50)
ds_pipe

### Applying transforms to pipelines adds more pipeline stages.
_Questions for clarification_:
 * _how can we send arguments to thse pipeline functions?_

In [ ]:
ds_pipe = ds_pipe.map(divide_row_value)
ds_pipe = ds_pipe.map(double_row_value)
ds_pipe = ds_pipe.map(modulo_row_value)
print(ds_pipe)

#### Iterate our pipeline

 * _Questions for clearification_:
     * _how is this executed_?
     * _why are we iterating over rows_?
     * _what is row comprised of? Blocks?_?
     * _is the value of the row an already computed value_?
     * _if the `num_stages=5`, why am I seeing on stages?_

In [ ]:
results=[]
for row in ds_pipe.iter_rows():
    results.append(row)
print(f"Total value: {sum(results)}")

Let's try a Datapipeline with our synthetic data

In [ ]:
# count or return based on the condition
def count_ca(row: ray.data.impl.arrow_block.ArrowRow) -> int:
    return 1 if row['state'] == "CA" and row["defaulted"] else 0

In [ ]:
arrow_ds_pipe = arrow_ds.window(blocks_per_window=50)
arrow_ds_pipe

In [ ]:
arrow_ds_pipe = arrow_ds_pipe.map(count_ca)
arrow_ds_pipe

In [ ]:
results=[]
for row in arrow_ds_pipe.iter_rows():
    results.append(row)
print(f"Total rows for CA state and defaulted loans rows: {sum(results)}")

### Ingesting into Model Trainers
Let's define a dummy trainer that takes our synthetic data and trains the model and returns loss for that trainer

In [ ]:
def model(input):
    return random.uniform(0, 1)

@ray.remote
class Trainer:
    def __init__(self, rank, model):
        self.rank = rank
        self.model = model
        self.loss = 0.0
        
    def train(self, shard:ray.data.Dataset) -> float:
        for batch in shard.iter_batches(batch_size=1024):
            for epoch in range(1,21):
                output = self.model(batch)
                self.loss = output 
        if epoch % 5 == 0:
            print(f'epoch {epoch}, loss: {self.loss:.3f}')
        return self.loss

In [ ]:
trainers = [Trainer.remote(i, model) for i in range(1, 6)]
trainers

In [ ]:
shards = arrow_ds.split(n=len(trainers), locality_hints=trainers)
shards

In [ ]:
object_refs = [t.train.remote(s) for t, s in zip(trainers, shards)]
ray.get(object_refs)

In [ ]:
ray.shutdown()

### Exercises
 1. Write some transformers, filters, and aggregators with our systhetic data
 2. Add additional pipleline stages with our synthetic data

### Homework

1. Work through the NYC example tutorial